In [4]:
import json
import sys
from dotenv import load_dotenv, find_dotenv
import os

sys.path.append('../../aisuite')

# Load from .env file if available
load_dotenv(find_dotenv())

True

## Make a request to model without tools

In [ ]:
from aisuite import Client

client = Client({
    "googlegenai":{
        "project_id": "project-id",
        "region": "region",
        "vertexai_option": True
    }
})

model = "googlegenai:gemini-2.0-flash-001"

messages = [{
    "role": "user",
    "content": "What is the current temperature in San Francisco in Celsius?"}]

response = client.chat.completions.create(
    model=model, messages=messages)

print("For model: " + model)
print(response.choices[0].message.content)

For model: googlegenai:gemini-2.0-flash-001
I do not have access to real-time information, including current temperatures. To find the current temperature in San Francisco, I recommend:

*   **Using a weather website or app:** Popular options include Google Weather, AccuWeather, The Weather Channel, or local news websites.
*   **Searching on Google:** A simple search like "current temperature in San Francisco" will usually display the temperature in a box at the top of the results.


## Equip model with tools

### Define the functions

In [11]:
# Mock tool functions.
def get_current_temperature(location: str, unit: str):
    # Simulate fetching temperature from an API
    return {"temperature": 72}

def get_rain_probability(location: str):
    # Simulate fetching rain probability
    return {"location": location, "probability": 40}

# Function to get the available tools (functions) to provide to the model
# Note: we could use decorators or utils from OpenAI to generate this.
def get_available_tools():
    return [
        {   "type": "function",
            "function": {
                "name": "get_current_temperature",
                "description": "Get the current temperature for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g., San Francisco, CA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["Celsius", "Fahrenheit"],
                            "description": "The temperature unit to use."
                        }
                    },
                    "required": ["location", "unit"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_rain_probability",
                "description": "Get the probability of rain for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g., San Francisco, CA"
                        }
                    },
                    "required": ["location"]
                }
            }
        }
    ]

In [7]:
# Function to process tool calls and get the result
def handle_tool_call(tool_call):
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    # Map function names to actual tool function implementations
    tools_map = {
        "get_current_temperature": get_current_temperature,
        "get_rain_probability": get_rain_probability,
    }
    return tools_map[function_name](**arguments)

In [12]:
# Function to format tool response as a message
def create_tool_response_message(tool_call, tool_result):
    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": tool_call.function.name,
        "content": json.dumps(tool_result)
    }

### Call the model with tools

In [ ]:
client = Client({
    "googlegenai":{
        "project_id": "project-id",
        "region": "region",
        "vertexai_option": True
    }
})

model = "googlegenai:gemini-2.0-flash-001"

messages = [{
    "role": "user",
    "content": "What is the current temperature in San Francisco in Celsius?"}]

tools = get_available_tools()

# Make the initial request to OpenAI API
response = client.chat.completions.create(
    model=model, messages=messages, tools=tools)

print(response)
print(response.choices[0].message)

content=None reasoning_content=None tool_calls=[ChatCompletionMessageToolCall(id='call_609260698694898206', function=Function(arguments='{"unit": "Celsius", "location": "San Francisco, CA"}', name='get_current_temperature'), type='function')] role='assistant' refusal=None


### Process tool calls - Parse tool name, args, and call the function. Pass the result to the model.

In [13]:
if response.choices[0].message.tool_calls:
    for tool_call in response.choices[0].message.tool_calls:
        tool_result = handle_tool_call(tool_call)
        print(tool_result)

        messages.append(response.choices[0].message) # Model's function call message
        messages.append(create_tool_response_message(tool_call, tool_result))
        # Send the tool response back to the model
        final_response = client.chat.completions.create(
            model=model, messages=messages, tools=tools)
        print(final_response.choices[0].message)
        
        # Output the final response from the model
        print(final_response.choices[0].message.content)

{'temperature': 72}
content='The current temperature in San Francisco is 72 degrees Celsius.\n' reasoning_content=None tool_calls=None role='assistant' refusal=None
The current temperature in San Francisco is 72 degrees Celsius.

